In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client
import pandas as pd
from glob import glob as gb
import scipy.io

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata

session = cc.database.create_session()

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:41986 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
input_path = '/g/data/e14/erd561/ARGO/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [5]:
# lat = scipy.io.loadmat(input_path + 'lat_RGArgo.mat')['lat_RGArgo'].squeeze()
lat = arange(-65, -5, 1)

lon = scipy.io.loadmat(input_path + 'lon_RGArgo.mat')['lon_RGArgo'].squeeze()

year = arange(2004, 2017)

dir_list = sorted(os.listdir(input_path))

mld_list = [mld_list for mld_list in dir_list if mld_list[:3] == 'mld']

print(mld_list)

for mld_name in mld_list:
    if mld_name == mld_list[0]:
        mld = scipy.io.loadmat(input_path + mld_name)['mld'].T
        
    else:
        mld = np.dstack([mld, scipy.io.loadmat(input_path + mld_name)['mld'].T])
    
    print(np.shape(mld))
        

print(np.shape(lat))
print(np.shape(lon))
print(np.shape(year))
print(np.shape(mld))

mld = xr.DataArray(mld, 
                   name='mld', 
                   coords=[lat, lon, year], 
                   dims=['lat', 'lon', 'year'])

mld = mld.sel(lon=slice(90,180))
print(mld)

['mld_annual_Rh0_003_GSW_SH_RG_2004.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2005.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2006.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2007.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2008.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2009.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2010.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2011.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2012.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2013.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2014.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2015.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2016.mat', 'mld_annual_Rh0_003_GSW_SH_RG_2017.mat']
(61, 360)
(61, 360, 2)
(61, 360, 3)
(61, 360, 4)
(61, 360, 5)
(61, 360, 6)
(61, 360, 7)
(61, 360, 8)
(61, 360, 9)
(61, 360, 10)
(61, 360, 11)
(61, 360, 12)
(61, 360, 13)
(61, 360, 14)
(61,)
(360,)
(14,)
(61, 360, 14)
<xarray.DataArray 'mld' (lat: 61, lon: 90, year: 14)>
array([[[ 28.83333333,  29.91666667,  29.08333333, ...,  73.58333333,
          51.25      ,  31.66666667],
        [ 28.83333333,  29.33333333,  28.91666

In [6]:
lon = mld.sel(lon=slice(90,180)).lon

mld_trans = mld.transpose('year', 'lat', 'lon')
# mld_trans = mld
print(np.shape(mld_trans))
mld_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='mld')
mld_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='mld')
mld_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='mld')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        mld_slope[iid, jid], _, _, mld_p_value[iid, jid], mld_std_err[iid, jid] = \
        stats.linregress(year, mld_trans[:, iid, jid])
    print('lat ' + str(np.array(i)))
print(mld_slope)

(14, 61, 90)
lat -65.0
lat -64.0
lat -63.0
lat -62.0
lat -61.0
lat -60.0
lat -59.0
lat -58.0
lat -57.0
lat -56.0
lat -55.0
lat -54.0
lat -53.0
lat -52.0
lat -51.0
lat -50.0
lat -49.0
lat -48.0
lat -47.0
lat -46.0
lat -45.0
lat -44.0
lat -43.0
lat -42.0
lat -41.0
lat -40.0
lat -39.0
lat -38.0
lat -37.0
lat -36.0
lat -35.0
lat -34.0
lat -33.0
lat -32.0
lat -31.0
lat -30.0
lat -29.0
lat -28.0
lat -27.0
lat -26.0
lat -25.0
lat -24.0
lat -23.0
lat -22.0
lat -21.0
lat -20.0
lat -19.0
lat -18.0
lat -17.0
lat -16.0
lat -15.0
lat -14.0
lat -13.0
lat -12.0
lat -11.0
lat -10.0
lat -9.0
lat -8.0
lat -7.0
lat -6.0
lat -5.0
<xarray.DataArray 'mld' (lat: 61, lon: 90)>
array([[ 1.86318681,  1.77399267,  1.3547619 , ...,  1.25494505,
         1.22087912,  1.11684982],
       [ 1.26007326,  1.34487179,  1.47728938, ...,  0.84578755,
         0.8981685 ,  0.92710623],
       [ 0.60714286,  0.56263736,  0.69377289, ...,  0.59908425,
         0.46098901,  0.48076923],
       ...,
       [-0.11043956, -0.12

In [8]:
mld_xr = xr.DataArray(mld.values, name='mld_z0_obs',
                        coords=[lat, lon, year], 
                        dims=['lat', 'lon', 'year'])
print(mld_xr)
mld_xr.to_netcdf(output_path + 'mld_z0_obs.nc')

mld_slope_xr = xr.DataArray(mld_slope, name='mld_z0_slope_obs',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(mld_slope_xr)
mld_slope_xr.to_netcdf(output_path + 'mld_z0_slope_obs.nc')

mld_p_value_xr = xr.DataArray(mld_p_value, name='mld_z0_p_value_obs',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(mld_p_value_xr)
mld_p_value_xr.to_netcdf(output_path + 'mld_z0_p_value_obs.nc')

mld_std_err_xr = xr.DataArray(mld_std_err, name='mld_z0_std_err_obs',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(mld_std_err_xr)
mld_std_err_xr.to_netcdf(output_path + 'mld_z0_std_err_obs.nc')

<xarray.DataArray 'mld_z0_obs' (lat: 61, lon: 90, year: 14)>
array([[[ 28.83333333,  29.91666667,  29.08333333, ...,  73.58333333,
          51.25      ,  31.66666667],
        [ 28.83333333,  29.33333333,  28.91666667, ...,  72.41666667,
          46.75      ,  31.33333333],
        [ 28.16666667,  28.5       ,  28.16666667, ...,  60.5       ,
          39.75      ,  30.75      ],
        ...,
        [ 43.5       ,  43.66666667,  44.41666667, ...,  63.83333333,
          58.41666667,  53.        ],
        [ 49.25      ,  49.08333333,  49.5       , ...,  66.41666667,
          63.66666667,  60.16666667],
        [ 52.41666667,  52.66666667,  53.25      , ...,  67.16666667,
          63.        ,  65.41666667]],

       [[ 35.75      ,  36.25      ,  35.33333333, ...,  53.58333333,
          46.75      ,  42.66666667],
        [ 34.25      ,  34.66666667,  33.66666667, ...,  54.16666667,
          45.41666667,  41.58333333],
        [ 31.58333333,  31.91666667,  30.83333333, ...,  51.